# MySQL AI

With HeatWave and OCI Generative AI, cloud customers can already build advanced generative AI applications powered by state-of-the-art large language models (LLMs) such as LLaMA 4, Grok, and OpenAI. These solutions are further enhanced through automated data ingestion with the HeatWave vector store and scalable AutoML, enabling a wide spectrum of use cases—from predicting delivery times to recommending personalized food choices.

MySQL AI extends these same generative AI and machine learning capabilities to smaller, air-gapped environments. This allows organizations to deploy intelligent solutions such as chatbots and enterprise search directly within MySQL installations that operate without cloud connectivity.

This notebook will showcase the use of [ML_RAG](https://dev.mysql.com/doc/heatwave/en/mys-hwgenai-ml-rag.html) for Retrieval Augmented Generation (RAG) and [HEATWAVE_CHAT](https://dev.mysql.com/doc/heatwave/en/mys-hwgenai-hw-chat.html) for engaging in natural language interactions using data from the 2024 Olympic Games.

### References
- https://www.economicsobservatory.com/what-happened-at-the-2024-olympics
- https://en.wikipedia.org/wiki/2024_Summer_Olympics

### Prerequistises
Install the necessary packages

- mysql-connector-python
- pandas 

##### Import Python packages

In [31]:
# import Python packages
import os
import json
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Connect to the HeatWave instance
We create a connection to an active [HeatWave](https://www.oracle.com/mysql/) instance using the [MySQL Connector/Python](https://dev.mysql.com/doc/connector-python/en/). We also define an API to execute a SQL query using a cursor, and the result is returned as a Pandas DataFrame. Modify the below variables to point to your HeatWave instance. On AWS, set USE_BASTION to False. On OCI, please create a tunnel on your machine using the below command by substituting the variable with their respective values

ssh -o ServerAliveInterval=60 -i BASTION_PKEY -L LOCAL_PORT:DBSYSTEM_IP:DBSYSTEM_PORT BASTION_USER@BASTION_IP

In [ ]:
BASTION_IP ="ip_address"
BASTION_USER = "opc"
BASTION_PKEY = "private_key_file"
DBSYSTEM_IP = "ip_address"
DBSYSTEM_PORT = 3306
DBSYSTEM_USER = "username"
DBSYSTEM_PASSWORD = "password"
DBSYSTEM_SCHEMA = "mlcorpus"
LOCAL_PORT = 3306
USE_BASTION = True

if USE_BASTION is True:
    DBSYSTEM_IP = "127.0.0.1"
else:
    LOCAL_PORT = DBSYSTEM_PORT

mydb = mysql.connector.connect(
    host=DBSYSTEM_IP,
    port=LOCAL_PORT,
    user=DBSYSTEM_USER,
    password=DBSYSTEM_PASSWORD,
    database=DBSYSTEM_SCHEMA,
    allow_local_infile=True,
    use_pure=True,
    autocommit=True,
)
mycursor = mydb.cursor()


# Helper function to execute SQL queries and return the results as a Pandas DataFrame
def execute_sql(sql: str) -> pd.DataFrame:
    mycursor.execute(sql)
    return pd.DataFrame(mycursor.fetchall(), columns=mycursor.column_names)


# ML_RAG operation

The available in-database LLMs have the limitation of being trained on public datasets. However, you can leverage MySQL AI to generate content based on your own proprietary data through Retrieval Augmented Generation (RAG). MySQL AI can convert your data into embeddings and store them in a vector store, and these embeddings can be used to provide your enterprise's specific context to the LLMs.

In [36]:
# Delete the vector_store_data_1 table if exists
execute_sql(f"""DROP TABLE if exists vector_store_data_1;""")

""


### Use VECTOR_STORE_LOAD to create a vector store from the files that contain the proprietary data.

To do that first copy the files into /var/lib/mysql-files folder

Example: sudo cp /home/john_doe/Olympics_2024.pdf /var/lib/mysql-files

In [37]:
execute_sql("""CALL sys.VECTOR_STORE_LOAD('file:///var/lib/mysql-files/2024_Summer_Olympics_Wikipedia.pdf', JSON_OBJECT("schema_name","mlcorpus","table_name","vector_store_data_1"))""")

""


Invoke the [ML_RAG](https://dev.mysql.com/doc/heatwave/en/mys-hwgenai-ml-rag.html) procedure with your query.

In [24]:
execute_sql(f"""CALL sys.ML_RAG("Where were the 2024 Summer Olympics held?", @output, NULL);""")
df = execute_sql(f"""SELECT JSON_PRETTY(@output);""")
json.loads(df.iat[0,0])["text"]

'The 2024 Summer Olympics were held in France.'

In [25]:
execute_sql(f"""CALL sys.ML_RAG("What were the opening and closing dates of the Games?", @output, NULL);""")
df = execute_sql(f"""SELECT JSON_PRETTY(@output);""")
json.loads(df.iat[0,0])["text"]

'The opening date of the 2024 Summer Olympics was July 26, 2024, and the closing date was August 11, 2024.'

In [26]:
execute_sql(f"""CALL sys.ML_RAG("Which sports made their Olympic debut in 2024?", @output, NULL);""")
df = execute_sql(f"""SELECT JSON_PRETTY(@output);""")
json.loads(df.iat[0,0])["text"]

'According to the text, one sport that made its Olympic debut in 2024 is "breaking" (also known as breakdancing).'

# HEATWAVE_CHAT operation

Thanks to the native support for vector store and Retrival Augmented Generation (RAG), you can load and query unstructured documents using natural language within the MySQL AI ecosystem.
This will be a brief demonstration of HeatWave Chat, a built-in chatbot that utilizes LLMs to understand and respond to your inputs.

Use VECTOR_STORE_LOAD to create a vector store from the files that contain the proprietary data (see previous section), and then, use the [HEATWAVE_CHAT](https://dev.mysql.com/doc/heatwave/en/mys-hwgenai-hw-chat.html) procedure to start a conversation with HeatWave.

In [27]:
df = execute_sql("""CALL sys.HEATWAVE_CHAT('What were the opening and closing dates of the Games?')""")
df["response"].iloc[0]

'The opening date of the 2024 Summer Olympics was July 26, 2024, and the closing date was August 11, 2024.'